In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

In [ ]:
img_rows,img_cols = 28,28
num_classes=10
num_epochs=30
batch_size = 64
learning_rate = 0.001

In [ ]:
def to_categorical(y,num_classes):
    return np.eye(num_classes,dtype = 'int64')[y]

In [ ]:
def data_prep(raw):
    out_y = to_categorical(raw.label,num_classes)
    num_imgs = raw.shape[0]
    x_array = raw.values[:,1:]
    x_shaped_array = x_array.reshape(num_imgs,1,img_rows,img_cols)
    out_x = x_shaped_array/255
    return out_x,out_y

In [ ]:
train_file = '../input/digit-recognizer/train.csv'
train_data = pd.read_csv(train_file)
x_train,y_train = data_prep(train_data)
#print(x.shape)
x_train=torch.Tensor(x_train)
y_train=torch.Tensor(y_train)

In [ ]:
import torch.utils.data as data_utils
train_set = data_utils.TensorDataset(x_train,y_train)

In [ ]:
n_total = len(train_data)
n_train = int(0.8*(n_total))
n_val = int(0.2*(n_total))

In [ ]:
import torch.utils.data as data_utils
# train_set = data_utils.TensorDataset(x_train,y_train)
#test_set = data_utils.TensorDataset(x_test,y_test)
train_set = data_utils.TensorDataset(x_train,y_train)
# train_set,val_set = data_utils.random_split(train_data_set,(n_train,n_val))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# dataloaders = {'train':data_utils.DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True),
#              'val':data_utils.DataLoader(dataset = val_set, batch_size = batch_size, shuffle = True)}
train_loader =data_utils.DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)

In [ ]:
# def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 scheduler.step()
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = int(labels)
#                 labels = labels.to(device)

#                 # zero the parameter gradients
#                 optimizer.zero_grad()

#                 # forward
#                 # track history if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)

#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 # statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             epoch_loss = running_loss / dataset_sizes[phase]
#             epoch_acc = running_corrects.double() / dataset_sizes[phase]

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                 phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model

In [ ]:
# def visualize_model(model, num_images=6):
#     was_training = model.training
#     model.eval()
#     images_so_far = 0
#     fig = plt.figure()

#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloaders['val']):
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)

#             for j in range(inputs.size()[0]):
#                 images_so_far += 1
#                 ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax.axis('off')
#                 ax.set_title('predicted: {}'.format(class_names[preds[j]]))
#                 imshow(inputs.cpu().data[j])

#                 if images_so_far == num_images:
#                     model.train(mode=was_training)
#                     return
#         model.train(mode=was_training)

In [ ]:
# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet,self).__init__()
#         self.layer1 = nn.Sequential(
#         nn.Conv2d(1,20,kernel_size=(3,3),stride=1,padding=1),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size=(2,2),stride=2))
#         self.layer2 = nn.Sequential(
#         nn.Conv2d(20,40,kernel_size=(3,3),stride=1,padding=1),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size=(2,2),stride=2))
#         self.drop_out = nn.Dropout()
#         self.fc1 = nn.Linear(40*7*7,128)
#         self.fc2 = nn.Linear(128,num_classes)
        
#     def forward(self,x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.reshape(out.size(0),-1)
#         out = self.drop_out(out)
#         out = self.fc1(out)
#         out = self.fc2(out)
#         return(out)

In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, 
            kernel_size=(7, 7), 
            stride=(2, 2), 
            padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(
            super(MnistResNet, self).forward(x), dim=-1)

In [ ]:
model_ft = MnistResNet()
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 10)

model = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
# scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.006,cycle_momentum=False)

In [ ]:
# model = ConvNet()
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.006,cycle_momentum=False)

In [ ]:
total_steps = len(train_loader)
loss_list = []
acc_list = []
lr_list = []
# j=0
for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(train_loader):
#         print(images.size())
#         learning_rate = 2.67e-7*j + 0.001
#         j+=1
        inputs = inputs.to(device)
        labels = labels.to(device)

        output = model(inputs)
        loss = criterion(output,torch.max(labels,1)[1])
        loss_list.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        total = labels.size(0)
        _,predicted = torch.max(output.data,1)
        labels = torch.max(labels,1)[1]
#         print(predicted.size())
#         print(labels.size())
        correct = (predicted == labels).sum().item()
#         acc_list.append(correct/total)
#         lr_list.append(learning_rate)
#         if (j+1) % 50 == 0:
#             acc_list.append(correct/total)
#             lr_list.append(learning_rate)
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                          (correct / total) * 100))
#lr_list.append(learning_rate)


In [ ]:
def test_prep(raw):
    num_imgs = raw.shape[0]
    x_array = raw.values[:,:]
    x_shaped_array = x_array.reshape(num_imgs,1,img_rows,img_cols)
    out_x = x_shaped_array/255
    return out_x

In [ ]:
model.eval()
test_file = '../input/digit-recognizer/test.csv'
test_data = pd.read_csv(test_file)
x_test = test_prep(test_data)
x_test = torch.Tensor(x_test)
x_test = x_test.to(device)
# test_loader = data_utils.DataLoader(dataset = x_test, batch_size = batch_size, shuffle = True)
y_pred = model(x_test)
pred = torch.max(y_pred,1)[1]
print(pred)

In [ ]:
print(len(pred))

In [ ]:
results = pd.Series(pred.cpu(),name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

In [ ]:
submission.to_csv("cnn_mnist_datagen.csv",index=False)